### Common Excel Tasks Demonstrated in Pandas
Why - 本文將介紹解析多個Excel文件、合併數據、清理數據和分析數據所需的基本流程

What - python 和 pandas 的結合對於這些活動來說是非常強大的，並且可以替代目前商業環境中經常使用的手動流程或痛苦的VBA腳本。切割和粘貼數據或編寫痛苦的VBA代碼很快就會過時。而 Python + pandas可以是一個很好的替代方案，它的可擴展性和功能更強。利用pandas處理Excel文件可以為你的數據處理需求開發一個更加精簡和可重複的解決方案。本文的其餘部分將展示一個簡單的例子，說明這個過程是如何進行的。我希望它能給你帶來如何將這些工具應用於你獨特情況的想法。

In [1]:
import pandas as pd
import numpy as np

In [2]:
import glob
glob.glob("data/sales*.xlsx")

['data\\sales-feb-2014.xlsx',
 'data\\sales-jan-2014.xlsx',
 'data\\sales-mar-2014.xlsx']

這就提供了我們所需要的東西。讓我們導入我們的每個文件並將它們合併成一個文件。熊貓的concat和append可以為我們做這個。在這個例子中我將使用append。

下面的代碼片段將初始化一個空白的DataFrame，然後將所有的單獨文件追加到all_data DataFrame中。

In [3]:
all_data = pd.DataFrame()
for f in glob.glob("data/sales*.xlsx"):
    df = pd.read_excel(f)
    all_data = all_data.append(df,ignore_index=True)

In [4]:
all_data.head()  #HDD process

,account number,name,sku,quantity,unit price,ext price,date
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32
3,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70,2014-02-01 19:56:48
4,672390,Kuhn-Gusikowski,S1-06532,48,55.82,2679.36,2014-02-02 03:45:20


In [5]:
all_data.describe()

,account number,quantity,unit price,ext price
count,384.000000,384.000000,384.000000,384.000000
mean,478125.989583,24.372396,56.651406,1394.517344
std,220902.947401,14.373219,27.075883,1117.809743
min,141962.000000,-1.000000,10.210000,-97.160000
25%,257198.000000,12.000000,32.612500,482.745000
50%,424914.000000,23.500000,58.160000,1098.710000
75%,714466.000000,37.000000,80.965000,2132.260000
max,786968.000000,49.000000,99.730000,4590.810000


In [6]:
all_data.dtypes

account number      int64
name               object
sku                object
quantity            int64
unit price        float64
ext price         float64
date               object
dtype: object

In [7]:
all_data['date'] = pd.to_datetime(all_data['date'])
all_data.dtypes

account number             int64
name                      object
sku                       object
quantity                   int64
unit price               float64
ext price                float64
date              datetime64[ns]
dtype: object

### Combining Data 合併數據
現在我們把所有的數據都放到一個DataFrame中，我們可以做DataFrame支持的任何操作。在這種情況下，我們要做的下一件事是讀入另一個文件，其中包含按賬戶劃分的客戶狀態。你可以把它看作是一個公司的客戶細分策略或其他一些識別客戶的機制。

In [8]:
status = pd.read_excel("data/saleMonth-customer-status.xlsx")
status.head()

,account number,name,status
0,740150,Barton LLC,gold
1,714466,Trantow-Barrows,silver
2,218895,Kulas Inc,bronze
3,307599,"Kassulke, Ondricka and Metz",bronze
4,412290,Jerde-Hilpert,bronze


我們想將這些數據與我們的銷售數據集合併。使用panda的合併函數，告訴它做一個左鍵連接，這類似於Excel的vlookup函數。

In [9]:
all_data_st = pd.merge(all_data, status, how='left')
all_data_st.head(3)

,account number,name,sku,quantity,unit price,ext price,date,status
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59,NaN
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46,bronze
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32,bronze


In [10]:
# Select an account
all_data_st[all_data_st["account number"]==737550].head()

,account number,name,sku,quantity,unit price,ext price,date,status
15,737550,"Fritsch, Russel and Anderson",S1-47412,40,51.01,2040.40,2014-02-05 01:20:40,NaN
25,737550,"Fritsch, Russel and Anderson",S1-06532,34,18.69,635.46,2014-02-07 09:22:02,NaN
66,737550,"Fritsch, Russel and Anderson",S1-27722,15,70.23,1053.45,2014-02-16 18:24:42,NaN
78,737550,"Fritsch, Russel and Anderson",S2-34077,26,93.35,2427.10,2014-02-20 18:45:43,NaN
80,737550,"Fritsch, Russel and Anderson",S1-93683,31,10.52,326.12,2014-02-21 13:55:45,NaN


這個賬戶號碼不在我們的狀態文件中，所以我們 status 有一堆NaN。我們可以決定如何處理這種情況。對於這個特定的情況，讓我們把所有缺失的帳戶標為某值(這邊使用銅級)。使用fillna函數可以很容易地在狀態欄上完成這個任務。

In [11]:
all_data_st['status'].fillna('bronze',inplace=True)
all_data_st.head()

,account number,name,sku,quantity,unit price,ext price,date,status
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59,bronze
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46,bronze
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32,bronze
3,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70,2014-02-01 19:56:48,bronze
4,672390,Kuhn-Gusikowski,S1-06532,48,55.82,2679.36,2014-02-02 03:45:20,silver


確認數據，現在我們已經填寫了所有數據以及狀態列。我們可以使用全套 pandas 功能進行正常的數據操作。

In [12]:
all_data_st[all_data_st["account number"]==737550].head()

,account number,name,sku,quantity,unit price,ext price,date,status
15,737550,"Fritsch, Russel and Anderson",S1-47412,40,51.01,2040.40,2014-02-05 01:20:40,bronze
25,737550,"Fritsch, Russel and Anderson",S1-06532,34,18.69,635.46,2014-02-07 09:22:02,bronze
66,737550,"Fritsch, Russel and Anderson",S1-27722,15,70.23,1053.45,2014-02-16 18:24:42,bronze
78,737550,"Fritsch, Russel and Anderson",S2-34077,26,93.35,2427.10,2014-02-20 18:45:43,bronze
80,737550,"Fritsch, Russel and Anderson",S1-93683,31,10.52,326.12,2014-02-21 13:55:45,bronze


### Categories 使用分類
pandas中一個相對較新的函數是對分類數據的支持。來自pandas的文檔。

分類是pandas的一種數據類型，對應於統計學中的分類變量：一個變量，它只能採取有限的，通常是固定的，可能的值（類別；R中的級別）。例如性別、社會階層、血型、國家歸屬、觀察時間或通過李克特量表的評分。
對於我們的目的，狀態字段是一個很好的類別類型的候選者。

In [13]:
all_data_st.dtypes

account number             int64
name                      object
sku                       object
quantity                   int64
unit price               float64
ext price                float64
date              datetime64[ns]
status                    object
dtype: object

In [14]:
all_data_st["status"] = all_data_st["status"].astype("category")
all_data_st.dtypes

account number             int64
name                      object
sku                       object
quantity                   int64
unit price               float64
ext price                float64
date              datetime64[ns]
status                  category
dtype: object

我們使用set_categories來告訴它我們要為這個類別對象使用的順序。在這種情況下，我們使用奧林匹克獎牌的排序。

In [15]:
all_data_st["status"].cat.set_categories([ "gold","silver","bronze"],inplace=True)
all_data_st.sort_values(["status"]).head()

C:\Users\cti110016\Anaconda3\lib\site-packages\pandas\core\arrays\categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


,account number,name,sku,quantity,unit price,ext price,date,status
68,740150,Barton LLC,B1-38851,17,81.22,1380.74,2014-02-17 17:12:16,gold
63,257198,"Cronin, Oberbrunner and Spencer",S1-27722,28,10.21,285.88,2014-02-15 17:27:44,gold
207,740150,Barton LLC,B1-86481,20,30.41,608.20,2014-01-22 16:33:51,gold
61,740150,Barton LLC,B1-20000,28,81.39,2278.92,2014-02-15 07:45:16,gold
60,239344,Stokes LLC,S2-83881,30,43.00,1290.00,2014-02-15 02:13:23,gold


### Analyze Data 分析數據
這個過程的最後一步是分析數據。現在，數據已被整合和清理，我們可以看看是否有任何可供學習的見解。

In [16]:
all_data_st["status"].describe()

count        384
unique         3
top       bronze
freq         172
Name: status, dtype: object

例如，如果你想快速查看你的頂級客戶與一般客戶的表現。使用groupby來獲得平均值。

In [17]:
all_data_st.groupby(["status"])["quantity","unit price","ext price"].mean()

C:\Users\CTI110~1\AppData\Local\Temp/ipykernel_18572/1444436240.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  all_data_st.groupby(["status"])["quantity","unit price","ext price"].mean()


,quantity,unit price,ext price
status,,,
gold,24.375000,53.723889,1351.944583
silver,22.842857,57.272714,1320.032214
bronze,25.616279,57.371163,1472.965930


In [18]:
all_data_st.groupby(["status"])["quantity","unit price","ext price"].agg([np.sum,np.mean, np.std])

C:\Users\CTI110~1\AppData\Local\Temp/ipykernel_18572/1265156672.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  all_data_st.groupby(["status"])["quantity","unit price","ext price"].agg([np.sum,np.mean, np.std])


quantity                       unit price                        \
            sum       mean        std        sum       mean        std   
status                                                                   
gold       1755  24.375000  14.575145    3868.12  53.723889  28.740080   
silver     3198  22.842857  14.512843    8018.18  57.272714  26.556242   
bronze     4406  25.616279  14.136071    9867.84  57.371163  26.857370   

        ext price                            
              sum         mean          std  
status                                       
gold     97340.01  1351.944583  1182.657312  
silver  184804.51  1320.032214  1086.384051  
bronze  253350.14  1472.965930  1116.683843

那麼，這說明了什麼？嗯，數據是完全隨機的，但我的第一個觀察是，我們賣給青銅客戶的單位比黃金客戶多。即使你看一下與青銅和黃金相關的總美元價值，我們賣給青銅客戶的數量比黃金多，這看起來很奇怪。也許我們應該看看我們有多少青銅客戶，看看到底發生了什麼？我打算做的是過濾掉獨特的賬戶，看看有多少黃金、白銀和青銅客戶。

In [19]:
all_data_st.drop_duplicates(subset=["account number","name"]).iloc[:,[0,1,7]].groupby(["status"])["name"].count()

status
gold      4
silver    7
bronze    9
Name: name, dtype: int64

好的。這就有點道理了。我們看到，我們有9個銅牌客戶，只有4個客戶。這可能是為什麼數量如此偏向於我們的銅牌客戶。鑑於我們將許多客戶默認為銅牌客戶的事實，這個結果是有道理的。也許我們應該把其中一些人重新分類？顯然，這個數據是假的，但希望這能說明你如何使用這些工具來快速分析你自己的數據。